In [1]:
cd("D:\\Home\\Git\\Oceananigans.jl\\")
pwd()

"D:\\Home\\Git\\Oceananigans.jl"

In [2]:
using Pkg
Pkg.activate(".")

"D:\\Home\\Git\\Oceananigans.jl\\Project.toml"

In [3]:
using Oceananigans, Oceananigans.Operators

In [4]:
g = RegularCartesianGrid((100, 100, 100), (10, 10, 10))

(Nx, Ny, Nz) = (100, 100, 100)
(Lx, Ly, Lz) = (10.0, 10.0, 10.0)
(Δx, Δy, Δz) = (0.1, 0.1, 0.1)

In [5]:
f1 = CellField(g);
f2 = FaceFieldX(g);

In [6]:
using BenchmarkTools

In [ ]:
@benchmark δx!(g, f1, f2)

In [7]:
@inline incmod1(a, n) = a == n ? one(a) : a + 1
@inline decmod1(a, n) = a == 1 ? n : a - 1

function δ0!(g, A, B)
    for k in 1:g.Nz, j in 1:g.Ny, i in 1:g.Nx
      @inbounds B[i, j, k] = A[i, j, k] - A[decmod1(i, g.Nx), j, k]
    end
end

function δ0!!(g, A, B)
    for k in 1:g.Nz, j in 1:g.Ny, i in 1:g.Nx
      @inbounds B[i, j, k] = A[i, j, k] - A[i, j, k]
    end
end

function δ1!(g, f, δf)
    for k in 1:g.Nz, j in 1:g.Ny, i in 1:g.Nx
      @inbounds δf[i, j, k] = f[i, j, k] - f[decmod1(i, g.Nx), j, k]
    end
end

function δ2!(g, f, δf)
    for k in 1:g.Nz, j in 1:g.Ny, i in 1:g.Nx
      @inbounds δf.data[i, j, k] = f.data[i, j, k] - f.data[decmod1(i, g.Nx), j, k]
    end
end

function δ3!(g, f, δf)
    for k in 1:g.Nz, j in 1:g.Ny, i in 1:g.Nx
        setindex!(δf.data, f.data[i, j, k] - f.data[decmod1(i, g.Nx), j, k], i, j, k)
    end
end

δ3! (generic function with 1 method)

In [8]:
A = rand(Float64, size(g));
B = rand(Float64, size(g));
@benchmark δ0!(g, A, B)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.314 ms (0.00% GC)
  median time:      1.487 ms (0.00% GC)
  mean time:        1.631 ms (0.00% GC)
  maximum time:     8.081 ms (0.00% GC)
  --------------
  samples:          3050
  evals/sample:     1

In [ ]:
@benchmark δ0!!(g, A, B)

In [ ]:
@benchmark δ1(g, f1, f2)

In [ ]:
@benchmark δ2(g, f1, f2)

In [ ]:
@benchmark δ4(g, f1, f2)

In [ ]:
struct MyArray{T,G<:Grid{T}}
    data::Array{T}
    grid::G
end

In [ ]:
function MyArray(grid::Grid{T}) where T <: AbstractFloat
    sz = size(grid)
    data = zeros(T, sz)
    CellField(data, grid)
end

In [ ]:
C = MyArray(g);
D = MyArray(g);

In [ ]:
@benchmark δ2(g, C, D)

In [ ]:
@benchmark setindex!(f1.data, 12.5, 5, 6, 7)

In [ ]:
@benchmark δ4(g, f1, f2)

In [ ]:
struct MyArray2
    data::Array{AbstractFloat}
    grid::Grid
end

In [ ]:
function MyArray2(grid::Grid, T=Float64)
    sz = size(grid)
    data = zeros(T, sz)
    CellField(data, grid)
end

In [ ]:
E = MyArray2(g);
F = MyArray2(g);

In [ ]:
@benchmark δ1(g, E, F)

In [ ]:
@benchmark δ2(g, E, F)

In [ ]:
@benchmark δ4(g, E, F)

In [9]:
struct MyCellField
    data::Array{Float64,3}
    grid::Grid
end

In [10]:
function MyCellField(grid, T=Float64)
    sz = size(grid)
    data = zeros(T, sz)
    MyCellField(data, grid)
end

MyCellField

In [11]:
fC1, fC2 = MyCellField(g), MyCellField(g);

In [12]:
@benchmark δ2!(g, fC1, fC2)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.642 ms (0.00% GC)
  median time:      1.852 ms (0.00% GC)
  mean time:        1.925 ms (0.00% GC)
  maximum time:     4.090 ms (0.00% GC)
  --------------
  samples:          2586
  evals/sample:     1

In [13]:
@benchmark δ3!(g, fC1, fC2)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.128 ms (0.00% GC)
  median time:      4.633 ms (0.00% GC)
  mean time:        4.714 ms (0.00% GC)
  maximum time:     8.625 ms (0.00% GC)
  --------------
  samples:          1059
  evals/sample:     1

In [19]:
@inline Base.getindex(f::MyCellField, inds...) = Base.getindex(f.data, inds...)
@inline Base.setindex!(f::MyCellField, v, inds...) = Base.setindex!(f.data, v, inds...)
@benchmark δ1!(g, fC1, fC2)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.527 ms (0.00% GC)
  median time:      5.033 ms (0.00% GC)
  mean time:        5.121 ms (0.00% GC)
  maximum time:     8.150 ms (0.00% GC)
  --------------
  samples:          975
  evals/sample:     1

In [14]:
struct MyCellField2
    data::Array
    grid::Grid
end

In [43]:
function MyCellField2(grid, T=Float64)
    sz = size(grid)
    data = zeros(Float32, sz)
    MyCellField2(data, grid)
end

MyCellField2

In [20]:
fc2a, fc2b = MyCellField2(g), MyCellField2(g);

In [23]:
@benchmark δ2!(g, fc2a, fc2b)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.641 ms (0.00% GC)
  median time:      1.821 ms (0.00% GC)
  mean time:        1.881 ms (0.00% GC)
  maximum time:     3.121 ms (0.00% GC)
  --------------
  samples:          2646
  evals/sample:     1

In [44]:
g32 = RegularCartesianGrid((100, 100, 100), (10, 10, 10), Float32)
fc2a32, fc2b32 = MyCellField2(g32, Float32), MyCellField2(g32, Float32);

In [45]:
@benchmark δ2!(g, fc2a32, fc2b32)

BenchmarkTools.Trial: 
  memory estimate:  45.78 MiB
  allocs estimate:  3000000
  --------------
  minimum time:     129.685 ms (0.67% GC)
  median time:      133.059 ms (0.71% GC)
  mean time:        133.313 ms (0.75% GC)
  maximum time:     141.366 ms (0.62% GC)
  --------------
  samples:          38
  evals/sample:     1

In [50]:
@benchmark δ3!(g, fc2a32, fc2b32)

BenchmarkTools.Trial: 
  memory estimate:  45.78 MiB
  allocs estimate:  3000000
  --------------
  minimum time:     133.871 ms (0.66% GC)
  median time:      136.439 ms (0.68% GC)
  mean time:        137.066 ms (0.73% GC)
  maximum time:     147.059 ms (0.59% GC)
  --------------
  samples:          37
  evals/sample:     1

In [47]:
struct MyCellField3{T<:Array}
    data::T
    grid::Grid
end

In [70]:
function MyCellField3(grid, T=Float64)
    sz = size(grid)
    data = zeros(T, sz)
    MyCellField3{Array{T,3}}(data, grid)
end

MyCellField3

In [60]:
fc3a, fc3b = MyCellField3(g), MyCellField3(g);

In [61]:
@benchmark δ2!(g, fc3a, fc3b)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.641 ms (0.00% GC)
  median time:      1.809 ms (0.00% GC)
  mean time:        1.868 ms (0.00% GC)
  maximum time:     3.345 ms (0.00% GC)
  --------------
  samples:          2666
  evals/sample:     1

In [64]:
g32 = RegularCartesianGrid((100, 100, 100), (10, 10, 10), Float32)
fc3a32, fc3b32 = MyCellField3(g32, Float32), MyCellField3(g32, Float32);

In [79]:
@benchmark δ2!(g, fc3a32, fc3b32)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.473 ms (0.00% GC)
  median time:      1.632 ms (0.00% GC)
  mean time:        1.721 ms (0.00% GC)
  maximum time:     3.634 ms (0.00% GC)
  --------------
  samples:          2893
  evals/sample:     1

In [67]:
typeof(g32)

RegularCartesianGrid{Float32}

In [83]:
struct MyCellField4{T,G<:Grid{T}} <: Field{G}
    data::AbstractArray{T}
    grid::G
end

In [77]:
function MyCellField4(grid, T=Float64)
    sz = size(grid)
    data = zeros(T, sz)
    MyCellField4{T,RegularCartesianGrid{T}}(data, grid)
end

MyCellField4

In [78]:
fc4a, fc4b = MyCellField4(g), MyCellField4(g);

In [80]:
@benchmark δ2!(g, fc4a, fc4b)

BenchmarkTools.Trial: 
  memory estimate:  45.78 MiB
  allocs estimate:  3000000
  --------------
  minimum time:     121.186 ms (0.80% GC)
  median time:      125.157 ms (0.81% GC)
  mean time:        125.190 ms (0.86% GC)
  maximum time:     133.520 ms (0.71% GC)
  --------------
  samples:          40
  evals/sample:     1

In [86]:
struct MyCellField6{T<:AbstractFloat}
    data::Array{T}
    grid::Grid
end

In [87]:
function MyCellField6(grid, T=Float64)
    sz = size(grid)
    data = zeros(T, sz)
    MyCellField6{T}(data, grid)
end

MyCellField6

In [91]:
fc6a, fc6b = MyCellField6(g), MyCellField6(g);

In [92]:
@benchmark δ2!(g, fc6a, fc6b)

BenchmarkTools.Trial: 
  memory estimate:  45.78 MiB
  allocs estimate:  3000000
  --------------
  minimum time:     114.667 ms (0.86% GC)
  median time:      120.158 ms (0.84% GC)
  mean time:        122.610 ms (0.89% GC)
  maximum time:     149.093 ms (0.83% GC)
  --------------
  samples:          41
  evals/sample:     1

In [93]:
struct MyCellField7{T<:AbstractArray}
    data::T
    grid::Grid
end

In [96]:
function MyCellField7(grid, T=Float64, dim=3)
    sz = size(grid)
    data = zeros(T, sz)
    MyCellField7{Array{T,dim}}(data, grid)
end

MyCellField7

In [97]:
fc7a, fc7b = MyCellField7(g), MyCellField7(g);

In [115]:
Base.getindex(f::MyCellField7, inds...) = Base.getindex(f.data, inds...)
Base.setindex!(f::MyCellField7, v, inds...) = Base.setindex!(f.data, v, inds...)
@benchmark δ1!(g, fc7a, fc7b)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.531 ms (0.00% GC)
  median time:      5.073 ms (0.00% GC)
  mean time:        5.282 ms (0.00% GC)
  maximum time:     15.275 ms (0.00% GC)
  --------------
  samples:          945
  evals/sample:     1

In [102]:
@benchmark δ2!(g, fc7a, fc7b)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.632 ms (0.00% GC)
  median time:      1.833 ms (0.00% GC)
  mean time:        1.910 ms (0.00% GC)
  maximum time:     3.778 ms (0.00% GC)
  --------------
  samples:          2606
  evals/sample:     1

In [104]:
size(g)

(100, 100, 100)